<a href="https://colab.research.google.com/github/Manish19629/Algorithms/blob/main/algo_yenKshortestPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import nbformat

# Define the path to the file in Google Drive
file_path = '/content/drive/MyDrive/shortest distance/Networks.ipynb'

# Load the notebook
with open(file_path) as f:
    notebook = nbformat.read(f, as_version=4)

# Execute the notebook cells to load the networks into the current session
exec("\n".join(cell['source'] for cell in notebook.cells if cell.cell_type == 'code'))


In [18]:
import networkx as nx

# Select the network and set up the graph in NetworkX
# Mesh Networks: mesh
# Torus Networks: torus
# Gaussian Networks: gaussian | Gaussian8Plus9 | Gaussian8Plus11 | Gaussian9Plus9 | Gaussian8Plus10 | Gaussian7Plus13
# EJ Networks: EJ8Plus9 | EJ7Plus10 | EJ5Plus12 | EJ9Plus9 | EJ5Plus13
# Cube-Connected Cycles Networks: CCC4D
# Hypercube Networks: HC5D
# Airport Dataset: Airport
# Airport Dataset: Maps150
# testNetwork = Maps150AA
# testNetwork = Maps250
# testNetwork = CCC4D
# testNetwork = gaussian
# testNetwork = Airport
selected_network = gaussian  # Change to your desired network (e.g., 'torus', 'gaussian')

G = nx.Graph()
G.add_nodes_from(selected_network.Nodes)
G.add_edges_from(selected_network.Edges)

In [19]:
from heapq import heappush, heappop
from copy import deepcopy

def yen_k_shortest_paths(G, source, destination, K):
    # List to store the K shortest paths
    paths = []

    # First, find the shortest path using BFS (as the graph is unweighted)
    try:
        shortest_path = nx.shortest_path(G, source=source, target=destination)
        shortest_path_length = len(shortest_path) - 1  # Path length is the number of edges
    except nx.NetworkXNoPath:
        print("No path found between source and destination.")
        return []

    paths.append((shortest_path_length, shortest_path))

    # List to store potential K-shortest paths
    potential_paths = []

    for k in range(1, K):
        for i in range(len(paths[k - 1][1]) - 1):
            spur_node = paths[k - 1][1][i]
            root_path = paths[k - 1][1][:i + 1]

            # Temporarily remove edges and nodes to simulate spur path
            removed_edges = []
            for path_cost, path in paths:
                if path[:i + 1] == root_path:
                    edge = (path[i], path[i + 1])
                    if G.has_edge(*edge):
                        removed_edges.append(edge)
                        G.remove_edge(*edge)

            # Find the spur path from spur_node to destination
            try:
                spur_path = nx.shortest_path(G, source=spur_node, target=destination)
                total_path = root_path[:-1] + spur_path
                total_path_length = len(total_path) - 1  # Unweighted path length
                heappush(potential_paths, (total_path_length, total_path))
            except nx.NetworkXNoPath:
                continue

            # Restore edges
            for edge in removed_edges:
                G.add_edge(*edge)

        # If there are no more potential paths, break
        if not potential_paths:
            break

        # Get the shortest path in the potential_paths heap
        new_path_length, new_path = heappop(potential_paths)
        paths.append((new_path_length, new_path))

    return [path for length, path in paths]

# Example of running the modified Yen's K-Shortest Paths on the selected network
# beginDiscovery(G, '0,0','7,2')
# beginDiscovery(G, 'N0','N-1-3i')
# beginDiscovery(G, 'DVO','DEL')
#beginDiscovery(G, '11267375817','11267375817')
source = 'N0'  # Define source based on the network
destination = 'N-1-3i'  # Define destination based on the network
K = 3  # Define the number of shortest paths you want

# Execute Yen's K-Shortest Paths
k_shortest_paths = yen_k_shortest_paths(G, source, destination, K)
print("K-Shortest Paths:", k_shortest_paths)


K-Shortest Paths: [['N0', 'N-1', 'N-1-i', 'N-1-2i', 'N-1-3i'], ['N0', 'N-i', 'N-1-i', 'N-1-2i', 'N-1-3i'], ['N0', 'N-i', 'N-2i', 'N-1-2i', 'N-1-3i']]
